In [1]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

import catboost
from catboost import CatBoostRegressor, CatBoostClassifier
import numpy as np
import optuna
from optuna.integration import CatBoostPruningCallback
from sklearn.metrics import accuracy_score

from prophet import Prophet
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df_1 = pd.read_csv('SPFB.RTS_200601_210119 (1).csv', sep=';')
df_2 = pd.read_csv('SPFB.RTS_210119_220621.csv', sep=';')
df_3 = pd.read_csv('SPFB.RTS_220621_221220.csv', sep=';')
df = df_1.append(df_2)
df = df.append(df_3)

#df = pd.read_csv('SPFB.RTS_200601_211231.csv', sep=';')
df = df.drop_duplicates()
df.columns = ['ticker', 'period', 'date', 'time', 'open', 'high', 'low', 'close', 'vol']
df = df.drop(['ticker', 'period'], axis=1)
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), format='%Y%m%d %H%M%S')
df = df.drop(['date', 'time'], axis=1)
df.set_index('datetime', inplace=True)
df = df.sort_index(ascending=True)

#TODO 5min, 15min, 30min???, to classification because +- itis important(maybee regression for measuring strenght of prediction) 
# (1)-buy: open - high < (open - low)*2; (-1)-sell: open - low > -(open - high)*2?, and test not 15min - 5min 5min 5min

In [4]:
df.head()

,open,high,low,close,vol
datetime,,,,,
2020-06-01 10:00:00,123240.0,124670.0,123240.0,124620.0,18919
2020-06-01 10:05:00,124620.0,124680.0,124360.0,124430.0,10012
2020-06-01 10:10:00,124450.0,124490.0,123780.0,123820.0,13818
2020-06-01 10:15:00,123810.0,124050.0,123630.0,123990.0,11081
2020-06-01 10:20:00,123980.0,124380.0,123980.0,124370.0,7528


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120874 entries, 2020-06-01 10:00:00 to 2022-12-20 21:55:00
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    120874 non-null  float64
 1   high    120874 non-null  float64
 2   low     120874 non-null  float64
 3   close   120874 non-null  float64
 4   vol     120874 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 5.5 MB


In [6]:
#df = df.loc['2022-08-01':'2022-12-01']
#df.loc[(df.index.hour >= 10) & (df.index.hour <= 13)] # Add dt in front to get the hour

In [7]:
def make_features(df, max_lag=10, step_lag=1, rolling_mean_fast=5, rolling_mean_slow=24, rolling_min_max_size=12, lag_neg=10):
    """Generate new features for timeseries df.

    Keyword arguments:
    df -- pandas.DataFrame
    max_lag -- max lag, start from 2 because using diff (default 6)
    step_lag -- step for lag (default 2)
    rolling_mean_fast -- rolling mean length (default 5)
    rolling_mean_slow -- rolling mean length (default 24)
    rolling_min_max_size -- rolling min and max (default 12)
    """
    df['hour'] = df.index.hour
    #df['month'] = df.index.month
    #df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek
    df['diff_price'] = df['close'].shift(1) - df['open'].shift(1)
    #df['diff_risk_buy'] = (df['high'] - df['open']) / ((df['open'] - df['low'])*2)
    #df['diff_risk_sell'] = (df['open'] - df['low']) / ((df['high'] - df['open'])*2)
    df['rolling_max_day'] = df.close.shift(1).rolling('10h').max()
    df['rolling_min_day'] = df.close.shift(1).rolling('10h').min()
    df['rolling_max_2day'] = df.close.shift(1).rolling('34h').max()
    df['rolling_min_2day'] = df.close.shift(1).rolling('34h').min()

    df['dif_close_rollmin'] = df.close.shift(1) - df['rolling_min_day']
    df['dif_close_rollmax'] = df.close.shift(1) - df['rolling_max_day']


#############################################################################
    # for lag in range(1, max_lag, step_lag):                              ###### 1
    #     df['close_lag_' + str(lag)] = df.close.shift(lag)

    # for lag in range(2, max_lag, step_lag):
    #     df['open_lag_' + str(lag)] = df.open.shift(lag)

    for lag in range(1, 4, step_lag):
        df['low_lag_' + str(lag)] = df.low.shift(lag)

    for lag in range(1, 4, step_lag):
        df['high_lag_' + str(lag)] = df.high.shift(lag)
    
    for lag in range(1, lag_neg, step_lag):
        df['low_lagneg_' + str(lag)] = df.low.shift(-lag)

    for lag in range(1, lag_neg, step_lag):
        df['high_lagneg_' + str(lag)] = df.high.shift(-lag)    

    # for lag in range(2, max_lag, step_lag):
    #     df['vol_lag_' + str(lag)] = df.vol.shift(lag)
############################################################################
    for lag in range(2, max_lag, 2):
        df['close_lag_min_diff_' + str(lag)] = df.close.shift(lag) - df['rolling_min_day']
        df['close_lag_max_diff_' + str(lag)] = df['rolling_max_day'] - df.close.shift(lag)
        df['close_lag_min_diff2d_' + str(lag)] = df.close.shift(lag) - df['rolling_min_2day']
        df['close_lag_max_diff2d_' + str(lag)] = df['rolling_max_2day'] - df.close.shift(lag)

    # for lag in range(2, max_lag, step_lag):
    #     df['open_lag_min_diff_' + str(lag)] = df.open.shift(lag) - df['rolling_min_day']
    #     df['open_lag_max_diff_' + str(lag)] = df['rolling_max_day'] - df.open.shift(lag)
    #     df['open_lag_min_diff2d_' + str(lag)] = df.open.shift(lag) - df['rolling_min_2day']
    #     df['open_lag_max_diff2d_' + str(lag)] = df['rolling_max_2day'] - df.open.shift(lag)

    # for lag in range(2, max_lag, step_lag):
    #     df['low_lag_min_diff_' + str(lag)] = df.low.shift(lag) - df['rolling_min_day']
    #     df['low_lag_max_diff_' + str(lag)] = df['rolling_max_day'] - df.low.shift(lag)
    #     df['low_lag_min_diff2d_' + str(lag)] = df.low.shift(lag) - df['rolling_min_2day']
    #     df['low_lag_max_diff2d_' + str(lag)] = df['rolling_max_2day'] - df.low.shift(lag)

    # for lag in range(2, max_lag, step_lag):
    #     df['high_lag_min_diff_' + str(lag)] = df.high.shift(lag) - df['rolling_min_day']
    #     df['high_lag_max_diff_' + str(lag)] = df['rolling_max_day'] - df.high.shift(lag)
    #     df['high_lag_min_diff2d_' + str(lag)] = df.high.shift(lag) - df['rolling_min_2day']
    #     df['high_lag_max_diff2d_' + str(lag)] = df['rolling_max_2day'] - df.high.shift(lag)

    for lag in range(1, 5, 1):
        df['vol_lag_min_diff_' + str(lag)] = df.vol.shift(lag)





    # df['lag_24h'] = df.close.shift(24)
    # #df['lagv_24h'] = df.vol.shift(24)


    df['std'] = df.close.shift(1).rolling(rolling_mean_fast).std()
    #df['stdv'] = df.vol.rolling(rolling_mean_fast).std()

    #df['rolling_mean_fast'] = df.close.shift(1).rolling(rolling_mean_fast).mean()
    #df['rolling_mean_fastv'] = df.vol.shift(1).rolling(rolling_mean_fast).mean()

    #df['rolling_mean_slow'] = df.close.shift(1).rolling(rolling_mean_slow).mean()
    #df['rolling_mean_slowv'] = df.iloc[:, [1]].shift(1).rolling(rolling_mean_slow).mean()

    #df['rolling_max'] = df.close.shift(1).rolling(rolling_min_max_size).max()
    #df['rolling_maxv'] = df.iloc[:, [1]].shift(1).rolling(rolling_min_max_size).max()

    #df['rolling_min'] = df.close.shift(1).rolling(rolling_min_max_size).min()
    #df['rolling_minv'] = df.iloc[:, [1]].shift(1).rolling(rolling_min_max_size).min()

    #df['diff'] = df.iloc[:, [0]].diff()

    ### volatility 5days mean and volatility today ### 
    df['volatil_today'] = df.close.shift(1).rolling('10h').max() - df.close.rolling('10h').min()
    
    df['volatil_1d'] = df.close.shift(1).rolling('d').max() - df.close.shift(1).rolling('d').min()
    df['volatil_2d'] = df.close.shift(1, freq="D").rolling('d').max() - df.close.shift(1, freq="D").rolling('d').min().fillna(0)
    df['volatil_3d'] = df.close.shift(2, freq="D").rolling('d').max() - df.close.shift(2, freq="D").rolling('d').min().fillna(0)
    df['volatil_4d'] = df.close.shift(3, freq="D").rolling('d').max() - df.close.shift(3, freq="D").rolling('d').min().fillna(0)
    df['volatil_5d'] = df.close.shift(4, freq="D").rolling('d').max() - df.close.shift(4, freq="D").rolling('d').min().fillna(0)
    
    df['volatil_5d_median'] = df[['volatil_1d', 'volatil_2d', 'volatil_3d', 'volatil_4d', 'volatil_5d']].median(axis=1)
    

    return df

## TODO rolling volume, , how far to max/min, (close+max)/2 instead close, fractal and max == fractal or not
## TODO lag for some features

In [8]:
make_features(df)
df.head()

,open,high,low,close,vol,hour,dayofweek,diff_price,rolling_max_day,rolling_min_day,rolling_max_2day,rolling_min_2day,dif_close_rollmin,dif_close_rollmax,low_lag_1,low_lag_2,low_lag_3,high_lag_1,high_lag_2,high_lag_3,low_lagneg_1,low_lagneg_2,low_lagneg_3,low_lagneg_4,low_lagneg_5,...,close_lag_max_diff2d_2,close_lag_min_diff_4,close_lag_max_diff_4,close_lag_min_diff2d_4,close_lag_max_diff2d_4,close_lag_min_diff_6,close_lag_max_diff_6,close_lag_min_diff2d_6,close_lag_max_diff2d_6,close_lag_min_diff_8,close_lag_max_diff_8,close_lag_min_diff2d_8,close_lag_max_diff2d_8,vol_lag_min_diff_1,vol_lag_min_diff_2,vol_lag_min_diff_3,vol_lag_min_diff_4,std,volatil_today,volatil_1d,volatil_2d,volatil_3d,volatil_4d,volatil_5d,volatil_5d_median
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:00:00,123240.0,124670.0,123240.0,124620.0,18919,10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124360.0,123780.0,123630.0,123980.0,124110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 10:05:00,124620.0,124680.0,124360.0,124430.0,10012,10,0,1380.0,124620.0,124620.0,124620.0,124620.0,0.0,0.0,123240.0,NaN,NaN,124670.0,NaN,NaN,123780.0,123630.0,123980.0,124110.0,123940.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18919.0,NaN,NaN,NaN,NaN,190.0,0.0,NaN,NaN,NaN,NaN,0.0
2020-06-01 10:10:00,124450.0,124490.0,123780.0,123820.0,13818,10,0,-190.0,124620.0,124430.0,124620.0,124430.0,0.0,-190.0,124360.0,123240.0,NaN,124680.0,124670.0,NaN,123630.0,123980.0,124110.0,123940.0,123980.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10012.0,18919.0,NaN,NaN,NaN,800.0,190.0,NaN,NaN,NaN,NaN,190.0
2020-06-01 10:15:00,123810.0,124050.0,123630.0,123990.0,11081,10,0,-630.0,124620.0,123820.0,124620.0,123820.0,0.0,-800.0,123780.0,124360.0,123240.0,124490.0,124680.0,124670.0,123980.0,124110.0,123940.0,123980.0,124070.0,...,190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13818.0,10012.0,18919.0,NaN,NaN,800.0,800.0,NaN,NaN,NaN,NaN,800.0
2020-06-01 10:20:00,123980.0,124380.0,123980.0,124370.0,7528,10,0,180.0,124620.0,123820.0,124620.0,123820.0,170.0,-630.0,123630.0,123780.0,124360.0,124050.0,124490.0,124680.0,124110.0,123940.0,123980.0,124070.0,124450.0,...,800.0,800.0,0.0,800.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11081.0,13818.0,10012.0,18919.0,NaN,800.0,800.0,NaN,NaN,NaN,NaN,800.0


In [9]:
df.columns

Index(['open', 'high', 'low', 'close', 'vol', 'hour', 'dayofweek', 'diff_price', 'rolling_max_day', 'rolling_min_day', 'rolling_max_2day', 'rolling_min_2day', 'dif_close_rollmin', 'dif_close_rollmax', 'low_lag_1', 'low_lag_2', 'low_lag_3', 'high_lag_1', 'high_lag_2', 'high_lag_3', 'low_lagneg_1', 'low_lagneg_2', 'low_lagneg_3', 'low_lagneg_4', 'low_lagneg_5', 'low_lagneg_6', 'low_lagneg_7', 'low_lagneg_8', 'low_lagneg_9', 'high_lagneg_1', 'high_lagneg_2', 'high_lagneg_3', 'high_lagneg_4', 'high_lagneg_5', 'high_lagneg_6', 'high_lagneg_7', 'high_lagneg_8', 'high_lagneg_9', 'close_lag_min_diff_2', 'close_lag_max_diff_2', 'close_lag_min_diff2d_2', 'close_lag_max_diff2d_2', 'close_lag_min_diff_4', 'close_lag_max_diff_4', 'close_lag_min_diff2d_4', 'close_lag_max_diff2d_4', 'close_lag_min_diff_6', 'close_lag_max_diff_6', 'close_lag_min_diff2d_6', 'close_lag_max_diff2d_6', 'close_lag_min_diff_8', 'close_lag_max_diff_8', 'close_lag_min_diff2d_8', 'close_lag_max_diff2d_8',
       'vol_lag_min_d

In [10]:
def test(high, low_neg, high_neg, close, day_max):
    if ((high.max() - close + 100)*2) > (close - low_neg.min()):
        if (high_neg.max() + 100) < high.max():
            if high.max() >= day_max:
                return 1

In [11]:
df['test'] = df.apply(lambda df: test(
    df[['high_lag_1', 'high_lag_2', 'high_lag_3']], 
    df[['low_lagneg_1', 'low_lagneg_2', 'low_lagneg_3', 'low_lagneg_4', 'low_lagneg_5', 'low_lagneg_6', 'low_lagneg_7', 'low_lagneg_8', 'low_lagneg_9']],
    df[['high_lagneg_1', 'high_lagneg_2', 'high_lagneg_3', 'high_lagneg_4', 'high_lagneg_5', 'high_lagneg_6', 'high_lagneg_7', 'high_lagneg_8', 'high_lagneg_9']],
    df['close'],
    df['rolling_max_day']
    ),
axis=1)

In [12]:
df.test = df.test.fillna(0)
df.test = df.test.astype('int')

In [13]:
df.test.unique()

array([0, 1])

In [14]:
dfs = df.copy()

In [15]:
df = df.drop(
    [
        'volatil_1d', 
        'volatil_2d', 
        'volatil_3d', 
        'volatil_4d', 
        'volatil_5d', 
        'open', 
        'close', 
        'low', 
        'high', 
        'rolling_max_day', 
        'rolling_min_day',
        'rolling_max_2day',
        'rolling_min_2day',
        'high_lag_1', 
        'high_lag_2', 
        'high_lag_3', 
        'low_lagneg_1', 
        'low_lagneg_2', 
        'low_lagneg_3', 
        'low_lagneg_4', 
        'low_lagneg_5', 
        'low_lagneg_6', 
        'low_lagneg_7', 
        'low_lagneg_8', 
        'low_lagneg_9', 
        'high_lagneg_1', 
        'high_lagneg_2', 
        'high_lagneg_3', 
        'high_lagneg_4', 
        'high_lagneg_5', 
        'high_lagneg_6', 
        'high_lagneg_7', 
        'high_lagneg_8', 
        'high_lagneg_9',
        'low_lag_1',
        'low_lag_2',
        'low_lag_3',
        'close'
        ], axis=1)
df.columns

Index(['vol', 'hour', 'dayofweek', 'diff_price', 'dif_close_rollmin', 'dif_close_rollmax', 'close_lag_min_diff_2', 'close_lag_max_diff_2', 'close_lag_min_diff2d_2', 'close_lag_max_diff2d_2', 'close_lag_min_diff_4', 'close_lag_max_diff_4', 'close_lag_min_diff2d_4', 'close_lag_max_diff2d_4', 'close_lag_min_diff_6', 'close_lag_max_diff_6', 'close_lag_min_diff2d_6', 'close_lag_max_diff2d_6', 'close_lag_min_diff_8', 'close_lag_max_diff_8', 'close_lag_min_diff2d_8', 'close_lag_max_diff2d_8', 'vol_lag_min_diff_1', 'vol_lag_min_diff_2', 'vol_lag_min_diff_3', 'vol_lag_min_diff_4', 'std', 'volatil_today', 'volatil_5d_median', 'test'], dtype='object')

In [16]:
# corr_matrix = df.corr().round(2)
    
# fig = px.imshow(corr_matrix,
#                 text_auto=True,
#                 width=800, 
#                 height=800,
#                 color_continuous_scale='Hot')
    
# fig.update_xaxes(side="top")
# fig.update_layout(title="Корреляционная матрица параметров")
# fig.show()

In [17]:
df = df.dropna()
df.head()

,vol,hour,dayofweek,diff_price,dif_close_rollmin,dif_close_rollmax,close_lag_min_diff_2,close_lag_max_diff_2,close_lag_min_diff2d_2,close_lag_max_diff2d_2,close_lag_min_diff_4,close_lag_max_diff_4,close_lag_min_diff2d_4,close_lag_max_diff2d_4,close_lag_min_diff_6,close_lag_max_diff_6,close_lag_min_diff2d_6,close_lag_max_diff2d_6,close_lag_min_diff_8,close_lag_max_diff_8,close_lag_min_diff2d_8,close_lag_max_diff2d_8,vol_lag_min_diff_1,vol_lag_min_diff_2,vol_lag_min_diff_3,vol_lag_min_diff_4,std,volatil_today,volatil_5d_median,test
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:40:00,5142,10,0,80.0,260.0,-540.0,200.0,600.0,200.0,600.0,550.0,250.0,550.0,250.0,0.0,800.0,0.0,800.0,800.0,0.0,800.0,0.0,3494.0,4769.0,6306.0,7528.0,156.620561,800.0,800.0,0
2020-06-01 10:45:00,10388,10,0,380.0,650.0,-150.0,260.0,540.0,260.0,540.0,390.0,410.0,390.0,410.0,170.0,630.0,170.0,630.0,610.0,190.0,610.0,190.0,5142.0,3494.0,4769.0,6306.0,189.868375,800.0,800.0,0
2020-06-01 10:50:00,9201,10,0,350.0,1000.0,0.0,650.0,350.0,650.0,350.0,200.0,800.0,200.0,800.0,550.0,450.0,550.0,450.0,0.0,1000.0,0.0,1000.0,10388.0,5142.0,3494.0,4769.0,328.709598,1000.0,1000.0,0
2020-06-01 10:55:00,3908,10,0,-80.0,910.0,-90.0,1000.0,0.0,1000.0,0.0,260.0,740.0,260.0,740.0,390.0,610.0,390.0,610.0,170.0,830.0,170.0,830.0,9201.0,10388.0,5142.0,3494.0,365.417569,1000.0,1000.0,0
2020-06-01 11:00:00,7206,11,0,80.0,1000.0,0.0,910.0,90.0,910.0,90.0,650.0,350.0,650.0,350.0,200.0,800.0,200.0,800.0,550.0,450.0,550.0,450.0,3908.0,9201.0,10388.0,5142.0,316.117067,1000.0,1000.0,0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120866 entries, 2020-06-01 10:40:00 to 2022-12-20 21:55:00
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   vol                     120866 non-null  int64  
 1   hour                    120866 non-null  int64  
 2   dayofweek               120866 non-null  int64  
 3   diff_price              120866 non-null  float64
 4   dif_close_rollmin       120866 non-null  float64
 5   dif_close_rollmax       120866 non-null  float64
 6   close_lag_min_diff_2    120866 non-null  float64
 7   close_lag_max_diff_2    120866 non-null  float64
 8   close_lag_min_diff2d_2  120866 non-null  float64
 9   close_lag_max_diff2d_2  120866 non-null  float64
 10  close_lag_min_diff_4    120866 non-null  float64
 11  close_lag_max_diff_4    120866 non-null  float64
 12  close_lag_min_diff2d_4  120866 non-null  float64
 13  close_lag_max_diff2d_4  120866 non-null 

In [19]:
dfsk = dfs.loc['2021-12-09':'2021-12-12 14:00']

In [20]:
import plotly.io as pio
pio.templates.default = "plotly_dark"

fig = go.Figure(data=[go.Candlestick(x=dfsk.index,
                open=dfsk['open'], high=dfsk['high'],
                low=dfsk['low'], close=dfsk['close'])
                     ])
# fig.add_trace(go.Scatter(x=dfs.index, 
#                    y=dfsk['rolling_max_day'],
#                    mode='lines',
#                    name='rolling_max', 
#                    fillcolor="#ffe476"))
# fig.add_trace(go.Scatter(x=dfs.index, 
#                    y=dfsk['rolling_min_day'],
#                    mode='lines',
#                    name='rolling_min',
#                    fillcolor="#00ff00"))

# fig.add_trace(go.Scatter(x=dfs.index, 
#                    y=dfsk['rolling_max_2day'],
#                    mode='lines',
#                    name='rolling_2max', 
#                    fillcolor="#ffe476"))
# fig.add_trace(go.Scatter(x=dfs.index, 
#                    y=dfsk['rolling_min_2day'],
#                    mode='lines',
#                    name='rolling_2min',
#                    fillcolor="#00ff00"))

fig.add_trace(go.Scatter(x=dfsk.index, 
                   y=dfsk['low'] * dfsk['test'] - 100,
                   mode='markers',
                   name='bs'))
fig.update_layout(xaxis_rangeslider_visible=True,
    width=1500,
    height=900,)
fig.show()

In [21]:
# df = df.loc['2022-08-01':'2022-12-01']
# df = df.loc[(df.index.hour >= 10) & (df.index.hour <= 13)] # Add dt in front to get the hour

In [22]:
# df.to_csv('tr.csv')

In [23]:
X = df.drop('test', axis=1)
y = df['test']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=False, test_size=0.1765)

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_val', X_val.shape)
print('y_val', y_val.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (84603, 29)
y_train (84603,)
X_val (18133, 29)
y_val (18133,)
X_test (18130, 29)
y_test (18130,)


In [29]:
tts = TimeSeriesSplit(n_splits=9)

pipe = Pipeline(
    [
        ("regressor", Ridge())
    ]
)

param_grid = [
    {
        'regressor': [CatBoostClassifier(random_state=47, 
                                        eval_metric='Accuracy', 
                                        silent=True,
                                        task_type='GPU',
                                        devices='0:1',
                                        early_stopping_rounds=10)],
        'regressor__depth': range(2, 16, 2),
        'regressor__learning_rate': np.linspace(0.1, 0.9, 20),
        'regressor__iterations': range(50,1500,50)
    },
    # {
    #     'regressor': [Ridge(random_state=47)],
    #     'regressor__alpha': [0.5, 1, 3, 5],
    #     'regressor__tol': [0.00001, 0.0001, 0.001],
    #     'regressor__max_iter': [4000, 8000, 10000],
    #     'regressor__solver': ['auto', 'cholesky', 'lsqr']
    # },
]

grid = RandomizedSearchCV(pipe, param_grid, n_iter=30 ,cv=tts, scoring = 'f1', verbose=3)

In [32]:
cat_features = ['hour', 'dayofweek']

In [41]:
from catboost import CatBoost

fast_model = CatBoostClassifier(
    random_seed=63,
    iterations=150,
    learning_rate=0.01,
    boosting_type='Plain',
    bootstrap_type='Bernoulli',
    subsample=0.5,
    one_hot_max_size=20,
    rsm=0.5,
    leaf_estimation_iterations=5,
    max_ctr_complexity=1,
    eval_metric='F1'
)

fast_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [43]:
tunned_model = CatBoostClassifier(
    random_seed=63,
    iterations=1000,
    learning_rate=0.03,
    l2_leaf_reg=3,
    bagging_temperature=1,
    random_strength=1,
    one_hot_max_size=2,
    leaf_estimation_method='Gradient',
    eval_metric='F1' 
)

tunned_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    verbose=False,
    eval_set=(X_test, y_test),
    plot=True

)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 

In [27]:
grid.fit(X_train, y_train)

Fitting 9 folds for each of 30 candidates, totalling 270 fits


KeyboardInterrupt: 

In [55]:
result = pd.DataFrame(grid.cv_results_)
result['mean_test_score'] = (result['mean_test_score']*-1)**0.5
result.sort_values('mean_test_score')[['mean_fit_time', 
                                       'std_fit_time',
                                       'mean_score_time', 
                                       'param_regressor', 
                                       'mean_test_score']].head()

,mean_fit_time,std_fit_time,mean_score_time,param_regressor,mean_test_score
0,0.032550,0.026768,0.000000,<catboost.core.CatBoostClassifier object at 0x...,NaN
1,0.023741,0.006814,0.000000,<catboost.core.CatBoostClassifier object at 0x...,NaN
2,7.396150,4.812866,0.015951,<catboost.core.CatBoostClassifier object at 0x...,NaN


In [86]:
y_val = grid.best_estimator_.predict(X_test)
y_val = np.where(y_val > 0.5, 1, 0)
test_val = mean_squared_error(y_test, y_val, squared=False)
print('RMSE работы модели на тестовой выборке:', round(test_val, 2))

RMSE работы модели на тестовой выборке: 0.16


In [89]:
final_result.head(30)

,buy_or_sell,pred
datetime,,
2022-11-14 13:10:00,0,0
2022-11-14 13:15:00,0,0
2022-11-14 13:20:00,0,0
2022-11-14 13:25:00,0,0
2022-11-14 13:30:00,1,0
2022-11-14 13:35:00,0,0
2022-11-14 13:40:00,0,0
2022-11-14 13:45:00,0,0
2022-11-14 13:50:00,0,0


In [88]:
final_result = pd.DataFrame(y_test)
final_result['pred'] = y_val

fig = px.line(final_result.loc['2022-11-11':'2022-11-30'])
fig.update_layout(title="Сравнение тестовой выборки и результата предсказания",
                  xaxis_title="Дата",
                  yaxis_title="Число закзов")
fig.show()

In [35]:
pred = tunned_model.predict(X_test)

In [37]:
from sklearn.metrics import f1_score

#pred = np.where(pred > 0.5, 1, 0)
f1_score(y_test, pred, average=None)

array([0.98413859, 0.1965812 ])

In [17]:
y.unique()

array([ 0, -1,  1], dtype=int64)

In [49]:
model = CatBoostClassifier(
    iterations=1000,
    random_seed=47,
    learning_rate=0.9,
    custom_metric='Logloss', #eval_metric='F1',
    use_best_model=True,
    early_stopping_rounds=50,
    )

In [50]:
model.fit(
    X_train,
    y_train,
    #cat_features=cat_features,
    eval_set=(X_val, y_val),
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [36]:
model.get_feature_importance(prettified=True).head(30)

,Feature Id,Importances
0,std,20.524290
1,dif_close_rollmax,16.073869
2,close_lag_max_diff_2,11.945945
3,close_lag_max_diff_4,9.449143
4,diff_price,4.242360
5,vol_lag_min_diff_2,4.142551
6,vol,2.858739
7,close_lag_min_diff_4,2.811396
8,dif_close_rollmin,2.748765
9,vol_lag_min_diff_3,2.639647


In [37]:
model.tree_count_

43

In [30]:
from catboost import cv
params = ()
params['loss_function'] = 'Logloss'


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 

In [39]:
pred = model.predict(X_test)

pd.DataFrame(pred)[0].unique()

array([0, 1], dtype=int64)

In [38]:
from sklearn.metrics import f1_score

#pred = np.where(pred > 0.5, 1, 0)
f1_score(y_test, pred)

0.1965811965811966

In [38]:
model.get_params()

{'iterations': 1000,
 'learning_rate': 0.2,
 'random_seed': 47,
 'use_best_model': True,
 'custom_metric': 'F1',
 'early_stopping_rounds': 50}

In [43]:
from sklearn import metrics, model_selection

In [44]:
predicted_valid = pred
fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted_valid)
fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={metrics.auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)

fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

In [ ]:
def objective(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'loss_function': 'RMSE',
        'task_type': 'GPU',
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        #'rsm': trial.suggest_uniform('rsm', 0.3, 1.0),
        #'subsample': trial.suggest_uniform('bagging_fraction', 0.4, 0.9),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.006, 0.018),
        'n_estimators':  25000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
    }
    model = CatBoostRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=200,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
pd.DataFrame(pred)[0].unique()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
model = DecisionTreeClassifier(class_weight='balanced')
model.fit(X_train, y_train)
pred = model.predict(X_test)
f1_score(y_test, pred, average=None)

In [ ]:
y.unique()

In [ ]:
tts = TimeSeriesSplit(n_splits=10)

pipe = Pipeline(
    [
        #('scaller', MinMaxScaler()),
        ("regressor", CatBoostClassifier())
    ]
)

param_grid = [
    {
        'regressor': [CatBoostClassifier(random_state=47, 
                                        #eval_metric='RMSE',
                                        loss_function='MultiClass', 
                                        silent=True, 
                                        early_stopping_rounds=5,
                                        task_type="GPU",
                                        devices='0:1',
                                        classes_count=6)],
        'regressor__depth': range(2, 12, 2),
        'regressor__learning_rate': np.linspace(0.3, 0.9, 7),
        'regressor__iterations': range(50,500,50)
    },
    # {
    #     'regressor': [Ridge(random_state=47)],
    #     'regressor__alpha': [0.5, 1, 3, 5],
    #     'regressor__tol': [0.00001, 0.0001, 0.001],
    #     'regressor__max_iter': [4000, 8000, 10000],
    #     'regressor__solver': ['auto', 'cholesky', 'lsqr']
    # },
]

grid = RandomizedSearchCV(pipe, param_grid, n_iter=10 ,cv=tts, verbose=3)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
result = pd.DataFrame(grid.cv_results_)
result['mean_test_score'] = (result['mean_test_score']*-1)**0.5
result.sort_values('mean_test_score')

In [ ]:
y_val = grid.best_estimator_.predict(X_test)
test_val = mean_squared_error(y_test, y_val, squared=False)
print('RMSE работы модели на тестовой выборке:', round(test_val, 2))

In [ ]:
final_result['buy_or_sell'].unique()

In [ ]:
final_result = pd.DataFrame(y_test)
final_result['pred'] = y_val

fig = px.line(final_result.loc['2021-12-01 10:30':'2021-12-15'])
fig.update_layout(title="Сравнение тестовой выборки и результата предсказания",
                  xaxis_title="date",
                  yaxis_title="pp")
fig.show()

In [ ]:
final_result.head(30)